# Menyelaraskan hiperparameter dengan pekerjaan pembersihan

Ada banyak algoritma pembelajaran mesin yang memerlukan hiperparameter (nilai parameter yang memengaruhi pelatihan, tetapi tidak dapat ditentukan dari data pelatihan itu sendiri). Misalnya, saat melatih model regresi logistik, Anda dapat menggunakan hiperparameter tingkat regularisasi untuk menangkal bias dalam model; atau saat melatih jaringan saraf konvolusional, Anda dapat menggunakan hiperparameter seperti tingkat pembelajaran dan ukuran batch untuk mengontrol bagaimana bobot disesuaikan dan berapa banyak item data yang diproses dalam batch mini masing-masing. Pilihan nilai hiperparameter dapat secara signifikan memengaruhi performa model terlatih, atau waktu yang diperlukan untuk melatihnya; dan seringkali Anda perlu mencoba beberapa kombinasi untuk menemukan solusi optimal. 

## Sebelum Anda memulai

Anda akan memerlukan versi terbaru paket  **azureml-ai-ml** untuk menjalankan kode di notebook ini. Jalankan sel di bawah ini untuk memverifikasi bahwa sel diinstal.

> **Catatan:**
> Jika paket **azure-ai-ml** tidak diinstal, jalankan `pip install azure-ai-ml` untuk menginstalnya.

In [ ]:
## Menyambungkan ke ruang kerja Anda

Dengan paket SDK yang diperlukan terinstal, sekarang Anda siap untuk terhubung ke ruang kerja Anda.

Untuk menyambungkan ke ruang kerja, kita memerlukan parameter pengidentifikasi - ID langganan, nama grup sumber daya, dan nama ruang kerja. Nama grup sumber daya dan nama ruang kerja sudah diisi untuk Anda. Anda hanya memerlukan ID langganan untuk menyelesaikan perintah.

Untuk menemukan parameter yang diperlukan, klik nama langganan dan ruang kerja di kanan atas Studio. Panel akan terbuka di sebelah kanan.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Salin ID langganan dan ganti **YOUR-SUBSCRIPTION-ID** dengan nilai yang Anda salin. </p>

## Membuat skrip pelatihan
Penyetelan hyperparameter sangat ideal ketika Anda ingin melatih model pembelajaran mesin tetapi bervariasi parameter input. Anda harus membuat skrip pelatihan yang mengharapkan parameter input yang mewakili salah satu hiperparameter algoritma.

Jalankan sel berikut untuk membuat folder **src** dan skrip pelatihan.

Perhatikan bahwa skrip pelatihan mengharapkan dua parameter input:

- `--training_data` yang mengharapkan string. Anda akan menentukan jalur ke aset data terdaftar sebagai data pelatihan input.
- `--reg_rate` yang mengharapkan angka, tetapi memiliki nilai `0.01`default . Anda akan menggunakan parameter input ini untuk penyetelan hiperparameter.

In [ ]:
## Mengonfigurasi dan menjalankan pekerjaan perintah

Jalankan sel di bawah ini untuk melatih model klasifikasi untuk memprediksi diabetes. Model ini dilatih dengan menjalankan skrip **py kereta\.** yang dapat ditemukan di folder **src**. Ini menggunakan aset data terdaftar `diabetes-data` sebagai data pelatihan. 

- `code`: menentukan folder yang menyertakan skrip yang akan dijalankan.
- `command`: menentukan apa yang harus dijalankan dengan tepat.
- `environment`: menentukan paket yang diperlukan untuk diinstal pada komputasi sebelum menjalankan perintah.
- `compute`: menentukan komputasi yang akan digunakan untuk menjalankan perintah.
- `display_name`: nama pekerjaan individu.
- `experiment_name`: nama eksperimen tempat pekerjaan berada.

Perhatikan bahwa pekerjaan perintah hanya menjalankan skrip pelatihan sekali, dengan tingkat `0.1`regularisasi . Sebelum Anda menjalankan pekerjaan pembersihan untuk menyelaraskan hiperparameter, ini adalah praktik terbaik untuk menguji apakah skrip Anda berfungsi seperti yang diharapkan dengan pekerjaan perintah.

In [ ]:
## Menentukan ruang pencarian

Ketika pekerjaan perintah Anda berhasil diselesaikan, Anda dapat mengonfigurasi dan menjalankan pekerjaan pembersihan. 

Pertama, Anda harus menentukan ruang pencarian untuk hyperparameter Anda. Untuk melatih tiga model, masing-masing dengan tingkat regularisasi yang berbeda (`0.01`, , atau `1`), Anda dapat menentukan ruang pencarian dengan `Choice` hyperparameter`0.1`. 

## Mengonfigurasi dan mengirimkan pekerjaan pembersihan

Anda akan menggunakan fungsi pembersihan untuk melakukan penyetelan hiperparameter pada skrip pelatihan Anda. Untuk mengonfigurasi pekerjaan pembersihan, Anda harus mengonfigurasi hal berikut:

- `compute`: Nama target komputasi untuk menjalankan pekerjaan.
- `sampling_algorithm`: Algoritma pengambilan sampel hyperparameter untuk digunakan di atas ruang pencarian. Nilai yang diizinkan adalah: `random`, `grid` dan `bayesian`.
- `primary_metric`: Nama metrik utama yang dilaporkan oleh setiap pekerjaan percobaan. Metrik harus dicatat dalam skrip pelatihan pengguna menggunakan `mlflow.log_metric()` dengan nama metrik yang sesuai.
- `goal`: Tujuan pengoptimalan dari `primary_metric`. Nilai yang diizinkan adalah `maximize` dan `minimize`.
- `limits`: Batasan untuk pekerjaan pembersihan. Misalnya, jumlah maksimum uji coba atau model yang ingin Anda latih.

Perhatikan bahwa pekerjaan perintah digunakan sebagai dasar untuk pekerjaan pembersihan. Konfigurasi untuk pekerjaan perintah akan digunakan kembali oleh pekerjaan pembersihan.

In [ ]:
Jalankan sel berikut untuk mengirimkan pekerjaan pembersihan.

In [ ]:
Ketika pekerjaan selesai, navigasikan ke ringkasan pekerjaan. Tab **Uji Coba** akan menampilkan semua model yang telah dilatih dan perbedaan `Accuracy` skor untuk setiap nilai tingkat regularisasi yang Anda coba.

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.